In [1]:
# Run this two lines only if running the very first time
# !pip install python-dotenv
# !pip install yfinance --upgrade

import os
import requests
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Scrap sp500 tickers using pandas datareader
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies") # Resource from wikipedia
ticker_table = tables[0]
tickers = ticker_table['Symbol'].tolist()

In [3]:
def get_data(ticker, start_year, end_year):
    
    start_year = start_year
    end_year = end_year + 1
    company_tick = ticker
    load_dotenv('.env_2')  # Load environment variables from the file '.env'
    API_KEY_FMP = os.environ.get('API_KEY_FMP')  # Retrieve the value of the environment variable 'API_KEY_FMP'
    API_KEY_FRED = os.environ.get('API_KEY_FRED')  # Retrieve the value of the environment variable 'API_KEY_FRED'

    # Construct API request endpoint url
    BASE_URL_FMP = 'https://financialmodelingprep.com/api/v3'
    endpoint_url_dividend = f"{BASE_URL_FMP}/historical-price-full/stock_dividend/{company_tick}?apikey={API_KEY_FMP}"
    endpoint_url_company_rating = f"{BASE_URL_FMP}/historical-rating/{company_tick}?apikey={API_KEY_FMP}"
    BASE_URL_FRED = 'https://api.stlouisfed.org/fred/series/observations'
    endpoint_url_interest_rate = f"{BASE_URL_FRED}?series_id=FEDFUNDS&api_key={API_KEY_FRED}&file_type=json&frequency=a&aggregation_method=avg&observation_start={start_year - 1}-01-01&observation_end={end_year - 1}-12-31"
    endpoint_url_inflation_rate = f"{BASE_URL_FRED}?series_id=CPIAUCSL&api_key={API_KEY_FRED}&file_type=json&frequency=a&aggregation_method=avg&observation_start={start_year - 1}-01-01&observation_end={end_year - 1}-12-31"


    # Send an HTTP GET request to the endpoint URL and store the response
    response = requests.get(endpoint_url_dividend)
    if response.status_code == 429:
        print("FMP API limit reached", {endpoint_url_dividend})
    elif response.status_code == 400:
        print("Invalid syntax", {endpoint_url_dividend})
    elif response.status_code == 403:
        print("Unauthorized access", {endpoint_url_dividend})
    elif response.status_code == 404:
        print("Can not find requested resources", {endpoint_url_dividend})
    elif response.status_code == 408:
        print("Request Timeout", {endpoint_url_dividend})
    elif response.status_code == 508:
        print("Loop Detected", {endpoint_url_dividend})
    elif response.status_code == 511:
        print("Network Authentication Required", {endpoint_url_dividend})
    import pandas as pd

    # Convert json to dictionary object and then a Pandas Dataframe
    response_dict = response.json()
    dividends = pd.DataFrame(response_dict['historical'])



    # Data Transformation
    if dividends.shape == (0, 0):  # Handle the case where the company never issued any dividend in the past
        dividends = pd.DataFrame({
            "year": list(range(start_year - 1, end_year + 1)),
            "adjDividend": [0.0] * len(list(range(start_year - 1, end_year + 1)))  # Obtaining 2 more years' data
        })
    else:
        # Extract year data from the date column
        dividends['year'] = pd.to_datetime(dividends['date']).dt.year
        # Aggregate the dividend paid by year
        dividends = dividends.groupby("year").agg({"adjDividend": "sum"}).reset_index()
        # Create a new DataFrame with all years from start to end - So that we don't omit years without dividends
        all_years = pd.DataFrame({'year': list(range(start_year - 1, end_year + 1))})
        # Merge the two DataFrames on the year column and fill missing values with 0.0
        dividends = all_years.merge(dividends, on='year', how='left').fillna(0.0)


    dividends['next_year_dividend'] = dividends['adjDividend'].shift(-1)

    conditions = [
        dividends['adjDividend'] <= dividends['next_year_dividend'],
        dividends['adjDividend'] > dividends['next_year_dividend']
    ]
    choices = ['constant/increased', 'decreased']

    # Create the target column 'dps_change' based on the conditions
    dividends['dps_change_next_year'] = np.select(conditions, choices, default=np.nan)

    # Construct dps change from last year
    dividends['last_year_dividend'] = dividends['adjDividend'].shift(1)
    dividends['dps_growth'] = dividends['adjDividend'] - dividends['last_year_dividend']

    # Another predictor that we can create is dividend change as a percentage
    dividends['dps_growth_rate'] = np.where(
        (dividends['last_year_dividend'] == 0) & (dividends['adjDividend'] == 0),
        0,  # If both are 0 then change is 0
        np.where(
            dividends['last_year_dividend'] != 0,
            ((dividends['adjDividend'] / dividends['last_year_dividend']) - 1) * 100,
            999  # If last year dividend is 0 then return 999
        )
    )

    # Remove the first last year since they will be NaN
    dividends = dividends.loc[(dividends['year'] >= start_year) & (dividends['year'] <= end_year - 1)]
    # Only keep the columns that we need
    dividends = dividends[["year", "adjDividend", "dps_growth", "dps_growth_rate", "dps_change_next_year"]]


    # Engineer some other predictors
    predictors = pd.DataFrame({"year": list(range(start_year - 1, end_year))})  # Include one more year before
                                                                                # the first year to calculate changes

    # Include the Company's Industry and sector data
    import yfinance as yf
    company_data_raw = yf.Ticker(company_tick)
    company_data = company_data_raw.info
    if 'industry' not in company_data:
        predictors['industry'] = 'NA'
        print(f"Company {company_tick} industry information is not available")
    else:
        predictors["industry"] = company_data['industry']
    if 'sector' not in company_data:
        predictors["sector"] = 'NA'
        print(f"Company {company_tick} sector information is not available")
    else:
        predictors["sector"] = company_data['sector']


    # Company Historical Rating
    # Send an HTTP GET request to the endpoint URL and store the response
    response = requests.get(endpoint_url_company_rating)
    if response.status_code == 429:
        print("FMP API limit reached", {endpoint_url_company_rating})
    elif response.status_code == 400:
        print("Invalid syntax", {endpoint_url_company_rating})
    elif response.status_code == 403:
        print("Unauthorized access", {endpoint_url_company_rating})
    elif response.status_code == 404:
        print("Can not find requested resources", {endpoint_url_company_rating})
    elif response.status_code == 408:
        print("Request Timeout", {endpoint_url_company_rating})
    elif response.status_code == 508:
        print("Loop Detected", {endpoint_url_company_rating})
    elif response.status_code == 511:
        print("Network Authentication Required", {endpoint_url_company_rating})

    # Convert json to dictionary object and then a Pandas Dataframe
    import pandas as pd
    response_dict = response.json()
    company_rating = pd.DataFrame(response_dict)


    # Data Transformation
    if company_rating.shape == (0, 0):  # Handle the case where the company has no rating in the past
        company_rating = pd.DataFrame({
            "year": list(range(start_year - 1, end_year + 1)),
            "ratingScore": [0.0] * len(list(range(start_year - 1, end_year + 1)))  # We are obtaining 2 more years' data
        })
    else:
        # Extract year data from the date column
        company_rating['year'] = pd.to_datetime(company_rating['date']).dt.year
        # Aggregate the rating by year
        company_rating = company_rating.groupby("year").agg({"ratingScore": "mean"}).reset_index()
        # Create a new DataFrame with all years from start to end - So that we don't omit years without rating
        all_years = pd.DataFrame({'year': list(range(start_year - 1, end_year + 1))})
        # Merge the two DataFrames on the year column and fill missing values with 0.0
        company_rating = all_years.merge(company_rating, on='year', how='left').fillna(0.0)
        predictors['companyRating'] = company_rating['ratingScore'].astype("float64")



    # Let's add some Macroeconomics indicators: the annualized Federal Interest Rate

    response = requests.get(endpoint_url_interest_rate)
    if response.status_code == 429:
        print("FRED API limit reached", {endpoint_url_interest_rate})
    elif response.status_code == 400:
        print("Invalid syntax", {endpoint_url_interest_rate})
    elif response.status_code == 403:
        print("Unauthorized access", {endpoint_url_interest_rate})
    elif response.status_code == 404:
        print("Can not find requested resources", {endpoint_url_interest_rate})
    elif response.status_code == 408:
        print("Request Timeout", {endpoint_url_interest_rate})
    elif response.status_code == 508:
        print("Loop Detected", {endpoint_url_interest_rate})
    elif response.status_code == 511:
        print("Network Authentication Required", {endpoint_url_interest_rate})
    data = response.json()

    fed_interest_rates = pd.DataFrame(data['observations'])
    predictors['interestRate'] = fed_interest_rates['value'].astype("float64")



    response = requests.get(endpoint_url_inflation_rate)
    if response.status_code == 429:
        print("FRED API limit reached", {endpoint_url_inflation_rate})
    elif response.status_code == 400:
        print("Invalid syntax", {endpoint_url_inflation_rate})
    elif response.status_code == 403:
        print("Unauthorized access", {endpoint_url_inflation_rate})
    elif response.status_code == 404:
        print("Can not find requested resources", {endpoint_url_inflation_rate})
    elif response.status_code == 408:
        print("Request Timeout", {endpoint_url_inflation_rate})
    elif response.status_code == 508:
        print("Loop Detected", {endpoint_url_inflation_rate})
    elif response.status_code == 511:
        print("Network Authentication Required", {endpoint_url_inflation_rate})
    data = response.json()

    # Convert data into DataFrame and convert values to float
    cpi_data = pd.DataFrame(data['observations'])
    predictors['inflationRate'] = cpi_data['value'].astype("float64")

    num_of_years = 2024 - start_year + 1

    response = requests.get(f"{BASE_URL_FMP}/ratios/{company_tick}?limit={num_of_years}&apikey={API_KEY_FMP}")
    if response.status_code == 429:
        print("FMP API limit reached", {endpoint_url_inflation_rate})
    elif response.status_code == 400:
        print("Invalid syntax", {endpoint_url_inflation_rate})
    elif response.status_code == 403:
        print("Unauthorized access", {endpoint_url_inflation_rate})
    elif response.status_code == 404:
        print("Can not find requested resources", {endpoint_url_inflation_rate})
    elif response.status_code == 408:
        print("Request Timeout", {endpoint_url_inflation_rate})
    elif response.status_code == 508:
        print("Loop Detected", {endpoint_url_inflation_rate})
    elif response.status_code == 511:
        print("Network Authentication Required", {endpoint_url_inflation_rate})


    # Check if all year's data is available
    data_length = len(response.json())
    if data_length != num_of_years:
        print(f"Company {company_tick} financial data is not available")
        return

    financial_ratios = pd.DataFrame(response.json()).iloc[:, :].sort_values("date", ascending=True).reset_index(drop=True)
    financial_ratios['calendarYear'] = financial_ratios['calendarYear'].astype('int64')
    predictors = predictors.merge(financial_ratios, left_on='year', right_on='calendarYear', how='left').fillna(0.0)

    # Drop unnecessary columns
    predictors.drop(["date", "calendarYear", "period"], axis="columns", inplace=True)



    def calculate_percentage_change(df, feature_name):
        percentage_change = df[feature_name].pct_change() * 100
        # Create new column name
        new_col_name = f"{feature_name}_percentage_change"
        # Find the index position of the original predictor column
        original_col_position = df.columns.get_loc(feature_name)
        # Insert the new column right after the original predictor column
        predictors.insert(original_col_position + 1, new_col_name, percentage_change)


    def calculate_actual_change(df, feature_name):
        actual_change = df[feature_name].diff()
        # Create new column name
        new_col_name = f"{feature_name}_actual_change"
        # Find the index position of the original predictor column
        original_col_position = df.columns.get_loc(feature_name)
        # Insert the new column right after the original predictor column
        predictors.insert(original_col_position + 2, new_col_name, actual_change)    

    feature_list = list(predictors.columns)

    feature_list.remove('year')
    feature_list.remove('industry')
    feature_list.remove('sector')
    feature_list.remove('symbol')



    for feature in feature_list:
        calculate_percentage_change(predictors, feature)
        calculate_actual_change(predictors, feature)



    # Replacing inf and NaN values
    predictors.replace([float('inf'), float('-inf')], 999, inplace=True)
    predictors.fillna(0, inplace=True)


    # Combine dividend data with other predictors
    dataset = pd.merge(dividends, predictors, left_on='year', right_on='year', how='left')

    # Move target to the end of the dataset for good practice
    feature_list = list(dataset.columns)
    feature_list.append('dps_change_next_year')
    feature_list.remove('dps_change_next_year')
    dataset = dataset[feature_list]
    return dataset

In [4]:
start_year = 2013
end_year = 2022


dataset = []
company_number = 1
for ticker in tickers:
    print(f"{company_number}: Obtaining data for {ticker}")
    company_number = company_number + 1
    company_data = get_data(ticker, start_year, end_year)
    if type(company_data).__name__ == "int":
        continue
    dataset.append(company_data)
dataset = pd.concat(dataset, ignore_index=True)

1: Obtaining data for MMM
2: Obtaining data for AOS
3: Obtaining data for ABT
4: Obtaining data for ABBV
5: Obtaining data for ACN
6: Obtaining data for ADBE
7: Obtaining data for AMD
8: Obtaining data for AES
9: Obtaining data for AFL
10: Obtaining data for A
11: Obtaining data for APD
12: Obtaining data for ABNB
Company ABNB financial data is not available
13: Obtaining data for AKAM
14: Obtaining data for ALB
15: Obtaining data for ARE
16: Obtaining data for ALGN
17: Obtaining data for ALLE
18: Obtaining data for LNT
19: Obtaining data for ALL
20: Obtaining data for GOOGL
21: Obtaining data for GOOG
22: Obtaining data for MO
23: Obtaining data for AMZN
24: Obtaining data for AMCR
25: Obtaining data for AEE
26: Obtaining data for AAL
27: Obtaining data for AEP
28: Obtaining data for AXP
29: Obtaining data for AIG
30: Obtaining data for AMT
31: Obtaining data for AWK
32: Obtaining data for AMP
33: Obtaining data for AME
34: Obtaining data for AMGN
35: Obtaining data for APH
36: Obtain

265: Obtaining data for JBHT
266: Obtaining data for JBL
267: Obtaining data for JKHY
268: Obtaining data for J
269: Obtaining data for JNJ
270: Obtaining data for JCI
271: Obtaining data for JPM
272: Obtaining data for JNPR
273: Obtaining data for K
274: Obtaining data for KVUE
Company KVUE financial data is not available
275: Obtaining data for KDP
276: Obtaining data for KEY
277: Obtaining data for KEYS
278: Obtaining data for KMB
279: Obtaining data for KIM
280: Obtaining data for KMI
281: Obtaining data for KLAC
282: Obtaining data for KHC
283: Obtaining data for KR
284: Obtaining data for LHX
285: Obtaining data for LH
286: Obtaining data for LRCX
287: Obtaining data for LW
Company LW financial data is not available
288: Obtaining data for LVS
289: Obtaining data for LDOS
290: Obtaining data for LEN
291: Obtaining data for LIN
292: Obtaining data for LYV
293: Obtaining data for LKQ
294: Obtaining data for LMT
295: Obtaining data for L
296: Obtaining data for LOW
297: Obtaining da

In [7]:
# Save data to disk
dataset.to_csv("Stock_data.csv", index=False)

In [11]:
dataset['adjDividend']

0       2.540
1       3.420
2       4.100
3       4.440
4       4.700
        ...  
4795    0.504
4796    0.656
4797    0.800
4798    1.000
4799    1.300
Name: adjDividend, Length: 4800, dtype: float64